In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `d:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests\Project.toml`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests\Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

MuJoCo Data object

In [5]:
println("number of geoms: ",model.ngeom)
finger_indices = [115,116,123,124,131,132,141,142,150,151]
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        115
        116
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        123
        124
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        131
        132
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        141
        142
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        150
        151
""";

number of geoms: 152
Simulation timestep: 0.005
Positions of joints: [-8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.000499746750576

## Test DIRCOL: moving index finger left right

In [6]:
tmax = 400
nx = model.nq + model.nv + model.na # State vector dimension
states = zeros(nx, tmax)
for t in 1:tmax
    states[:,t] = get_physics_state(model, data)
    step!(model, data)
end

In [7]:
function wrapped_mj_step(model, data, x, uk)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] = uk[:]
        # println("hi33")
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] = converted_uk[:]
        # println("hi33")
        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end

    # take discrete dynamics step 
    step!(model, data) 

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv)
    # xkp1 .= get_physics_state(model, data)
    return data.geom_xpos[115,:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        cost += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xt = Z[idx.x[N]]
    cost += 0.5*(xt-xg)'*Qf*(xt-xg)
    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    # get usual parameters from params
    idx, N, dt = params.idx, params.N, params.dt
    
    # get mujoco simulation needed params
    idx_full, xic_full = params.idx_full, params.xic_full
    model = params.model
    data = params.data


    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)

    # create the full state and initialize it
    x_u_full_for_passing = zeros(idx_full.nz)
    x_u_full_for_passing[1:(model.nq + model.nv + model.nu)] .= xic_full

    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        xip1 = Z[idx.x[i+1]]
        
        # use Mujoco as integrater
        xi_full = x_u_full_for_passing[idx_full.x[i]]
        ui_full = x_u_full_for_passing[idx_full.u[i]]
        xip1_mujoco, xi_1 = wrapped_mj_step(model, data, xi_full, ui_full)
        x_u_full_for_passing[idx_full.x[i+1]] .= xi_1
        # println("xip1_mujoco size:", size(xip1_mujoco))
        # println("xip1 size:", size(xip1))
        c[idx.c[i]] =  xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; robohand_dynamics_constraints(params,Z)]
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x = x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [8]:
function con(params, x)
    [
        robohand_equality_constraints(params, x);
        robohand_inequality_constraints(params, x)
    ]
end

# instantiate model and data
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

# initiate time and time steps
dt = 0.05
tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)

# LQR cost
# Q size only one because we track one finger
Q = 1
R = 0.1*diagm(ones(model.nu))
Qf = 10

# indexing 
# initialize index for state that only has the cartesian as states [x,y,z]
idx = create_idx(3, model.nu, N)
# initialize full index for full states
idx_full = create_idx(model.nq + model.nv, model.nu, N)

# calculate final state
start_pos = data.geom_xpos[115,:]
end_pos = data.geom_xpos[115,:] +  [0.05, 0.05, 0.0]
# initial and goal state
xic = start_pos
xg = end_pos

# initial full_state
xic_full = [data.qpos[:]; data.qvel[:]; data.ctrl[:]]

# initialize the first full states 
x_u_full_for_passing = zeros(idx_full.nz)

# x_u_full_for_passing[1:(model.nq + model.nv + model.nu)] .= [get_physics_state(model, data); data.ctrl[:]]


params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, idx_full=idx_full, xic_full=xic_full, model=model, data=data)

z0 = 0.001*randn(idx.nz)
println(size(z0))
ForwardDiff.gradient(_x -> robohand_cost(params, _x), z0)
# ForwardDiff.jacobian(_x -> robohand_dynamics_constraints(params, _x), z0)

(503,)


503-element Vector{Float64}:
 -0.04932203579754475
 -0.04986979245625313
 -0.0012349713696400025
  6.697953092396053e-5
 -0.00013438560262886128
  7.146950110023195e-5
 -0.00011525871527907015
 -8.550035827456707e-5
 -6.480911444413621e-5
 -0.00010721786538059044
  7.588979901270098e-5
  4.855172964340082e-6
  2.357946127328083e-5
  ⋮
  1.976692484476048e-5
 -5.9103604049627476e-5
  0.0001196042173015987
  0.00013604246798091678
 -8.282239884898933e-5
 -9.13911164823079e-7
 -4.882232578283046e-5
  2.605292963737015e-5
 -3.0228151277367585e-5
 -0.48294066369181887
 -0.49411107834722995
 -0.007333799996138405

In [9]:

dt = 0.05
tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)
nz = (N-1) * model.nu + N * (model.nq + model.nv)
x_u_full_for_passing = zeros(nz)
println(size([get_physics_state(model, data);data.ctrl[:]]))
x_u_full_for_passing[1:250] .= [get_physics_state(model, data);data.ctrl[:]]
println(nz)
println(model.nq + model.nv+model.nu)
println(size(x_u_full_for_passing))

(250,)
5228
250
(5228,)


In [10]:
size([data.qpos[:]; data.qvel[:]; data.ctrl[:]])

(250,)

In [35]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.05
    tf = 5.0
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size only 3 because we track one finger cartesian coordinates
    Q = 1*diagm(ones(3))
    R = 0.1*diagm(ones(model.nu))
    Qf = 100*diagm(ones(3))

    # indexing 
    # initialize index for state that only has the cartesian as states [x,y,z]
    idx = create_idx(3, model.nu, N)
    # initialize full index for full states
    idx_full = create_idx(model.nq + model.nv, model.nu, N)

    # calculate final state
    start_pos = data.geom_xpos[115,:]
    end_data = data.geom_xpos[115,:] +  [0.2, 0.2, 0.0]
    # initial and goal state
    xic = start_pos
    xg = end_data

    # initial full_state
    xic_full = [data.qpos[:]; data.qvel[:]; data.ctrl[:]]
    
    # initialize the first full states 
    x_u_full_for_passing = zeros(idx_full.nz)
    
    # x_u_full_for_passing[1:(model.nq + model.nv + model.nu)] .= [get_physics_state(model, data); data.ctrl[:]]


    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, idx_full=idx_full, xic_full=xic_full, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)

    for i =1:N-1
        x_l[idx.u[i]] .= -1
        x_u[idx.u[i]] .= 1
    end

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = 0.001*randn(idx.nz)
    z0[idx.x[1]] .= data.geom_xpos[115,:]
    z0[idx.u[1]] .= data.ctrl[:]
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
    # diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [36]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:   765918
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     2503
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2200
                     variables with only upper bounds:        0
Total number of equality constraints.................:      306
Total number of inequality constraints......

([[1.4640634438596227e-20, 1.4640634438596227e-20, 0.0], [0.0004900000000000147, 0.11699999999999999, 0.12000000000000001], [0.0004900356477847839, 0.11699984370778511, 0.11977426364260266], [0.0004908516299502036, 0.11699962753590754, 0.11938676273150908], [0.0004932517200174503, 0.11699941862742246, 0.11889258967498958], [0.0004978115117948712, 0.11699925138306297, 0.11833186138152817], [0.0005049005369356391, 0.116999155071389, 0.117730056102121], [0.000514654836096114, 0.1169991246199062, 0.11710743614116581], [0.0005270074216779524, 0.11699914480450516, 0.11647986858512827], [0.0005417326740004263, 0.1169992002258474, 0.11585967597117894]  …  [0.0009185470716845182, 0.11700049378592955, 0.1086803626903352], [0.0009189886809466905, 0.11700049145415861, 0.10867532313207423], [0.000919425117747695, 0.11700048911597652, 0.10867034634105416], [0.0009198564399488796, 0.11700048677173439, 0.10866543151359516], [0.0009202827048665901, 0.11700048442177644, 0.10866057785663027], [0.00092070

In [37]:
# instantiate model and data
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
N = length(t_vec)
reset!(model, data)
step!(model, data)
println(data.geom_xpos[115,:])
println(data.geom_xpos[115,:] +  [0.2, 0.2, 0.0])
states[:,1] = get_physics_state(model, data)
for t in 1:(N-1)
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println(data.geom_xpos[115,:])

[0.0004900000000000147, 0.11699999999999999, 0.12000000000000001]
[0.20049000000000003, 0.317, 0.12000000000000001]
[0.000922737216836382, 0.11700047022179254, 0.10863269737862886]


In [38]:
init_visualiser()
visualise!(model, data, trajectories = [states, ctrl_states])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [72]:
model, data = MuJoCo.sample_model_and_data()
function random_controller!(m::Model, d::Data)
    nu = m.nu
    d.ctrl .= 2*rand(nu) .- 1
    return nothing
end
reset!(model, data)
data.qpos[3] = 2
forward!(model, data) # Propagate the physics forward
tmax = 400
nx = model.nq + model.nv + model.na # State vector dimension
states = zeros(nx, tmax)
for t in 1:tmax
    states[:,t] = get_physics_state(model, data)
    step!(model, data)
end
reset!(model, data)
ctrl_states = zeros(nx, tmax)
for t in 1:tmax
    ctrl_states[:,t] = get_physics_state(model, data)
    random_controller!(model, data)
    step!(model, data)
end

In [75]:
init_visualiser()
visualise!(model, data, trajectories = [states, ctrl_states])

┌ Warning: The visualiser for MuJoCo.jl is designed to run multi-threaded, but the current Julia session was started with only one thread. Degraded performance will occur. To enable multi-threading, set JULIA_NUM_THREADS to a value greater than 1 before starting Julia.
└ @ VisualiserExt C:\Users\conan0223\.julia\packages\MuJoCo\9JERS\ext\VisualiserExt\VisualiserExt.jl:76


 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
